# Find common sites of the crawls and then push the list to the Mongodb


In [1]:
from consent.consistency.util import get_scan_root_dir, LOCATIONS
from consent.data.database.exper_result_database import ExperResultDatabase

import pandas as pd


In [2]:
cur_set = '0k_20k'
dfs = []
for location in LOCATIONS:
    SCAN_ROOT_DIR = get_scan_root_dir(location)
    site_compiles = pd.read_parquet(SCAN_ROOT_DIR / f'all_complies_{cur_set}.parquet')
    site_compiles['location'] = location
    dfs.append(site_compiles)
all_compiles = pd.concat(dfs)

In [3]:
global_find_sites = pd.DataFrame({'site': sorted(all_compiles.site.unique())})
print(f"Number of sites that are found to have cookie consent in at least 1 region: {len(global_find_sites)}")
global_find_sites.head()

Number of sites that are found to have cookie consent in at least 1 region: 1801


,site
0,247sports.com
1,2k.com
2,2ksports.com
3,9gag.com
4,aao.org


In [4]:
# Get the url
en_suc = ExperResultDatabase('2023-11-19', f'en_suc_{cur_set}').query_to_df()
en_suc

,site,url
0,amazonaws.com,https://aws.amazon.com/
1,google.com,https://www.google.com/
2,ntp.org,https://www.ntp.org/
3,googledomains.com,https://domains.google/
4,facebook.com,https://www.facebook.com/
...,...,...
10431,wrike.com,https://www.wrike.com/
10432,swwsite4.live,https://www.swwsite4.live/
10433,imgsed.com,https://imginn.com/
10434,genial.ly,https://genial.ly/


In [5]:
global_find = pd.merge(global_find_sites, en_suc, on='site', how='left')
assert len(global_find[global_find['url'].isna()]) == 0, 'All global-successful sites should be in the en_suc'
print(f"Number of global-successful sites: {len(global_find):,d}")
global_find.head()

Number of global-successful sites: 1,801


,site,url
0,247sports.com,https://247sports.com/
1,2k.com,https://www.2k.com/en-US/
2,2ksports.com,https://2k.com/en-US/
3,9gag.com,https://9gag.com/
4,aao.org,https://www.aao.org/


In [7]:
# Uncomment to write.
# ExperResultDatabase('2024-10-18', f'global_find_{cur_set}').write_df(global_find)

Written 1,801 records to db=consent_project-2024-10-18 collection=global_find_0k_20k
